<a href="https://colab.research.google.com/github/madixson/DogsvsCats_4380Project/blob/main/ResNetModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Build ResNet Model with pretrained parameters

In [ ]:
# Import necessary packages
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Set the data directories
train_dir = '/content/drive/MyDrive/DATA_4380/dataset_dogs_vs_cats/train/'
test_dir = '/content/drive/MyDrive/DATA_4380/dataset_dogs_vs_cats/test/'

# Set the image size and batch size
img_size = (224, 224)
batch_size = 32

# Use the ImageDataGenerator class to rescale the pixel values of the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of training and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Load pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine base model with new top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history_resnet = model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size,
          epochs=10, validation_data=test_generator, validation_steps=test_generator.n//test_generator.batch_size)


Found 18796 images belonging to 2 classes.
Found 6344 images belonging to 2 classes.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
587/587 [==============================] - 5619s 10s/step - loss: 0.6563 - accuracy: 0.6096 - val_loss: 0.6314 - val_accuracy: 0.6403
Epoch 2/10
587/587 [==============================] - 5263s 9s/step - loss: 0.6374 - accuracy: 0.6327 - val_loss: 0.6474 - val_accuracy: 0.6078
Epoch 3/10
587/587 [==============================] - 5147s 9s/step - loss: 0.6262 - accuracy: 0.6476 - val_loss: 0.6490 - val_accuracy: 0.6128
Epoch 4/10
587/587 [==============================] - 5073s 9s/step - loss: 0.6191 - accuracy: 0.6571 - val_loss: 0.5991 - val_accuracy: 0.6785
Epoch 5/10
587/587 [==============================] - 5066s 9s/step - loss: 0.6167 - accuracy: 0.6548 - val_loss: 0.6144 - val_accuracy: 0.6600
Epoch 6/10
587/587 [==============================] - 5189s 9s/step - loss: 0.6102 - accuracy: 0.6647 - val_loss: 0.5953 - val_ac

## Save results
Use pickle package to save training results for analysis result

In [6]:
import pickle

with open('/content/drive/MyDrive/DATA_4380/history_resnet.pkl', 'wb') as f:
    pickle.dump(history_resnet.history, f)